# くずし字 MNIST データセットの学習
[github](https://github.com/rois-codh/kmnist)からダウンロードしたくずし字データセットを今回作成したニューラルネットワークの分類タスク用クラスClassificationを用いて学習させ、その精度を求めた。

In [1]:
import numpy as np
from neural import Classification #自作ライブラリ
import matplotlib.pyplot as plt

In [2]:
x_train = np.load('./datasets/kmnist-train-imgs.npz')['arr_0']
t_train = np.load('./datasets/kmnist-train-labels.npz')['arr_0']
x_test = np.load('./datasets/kmnist-test-imgs.npz')['arr_0']
t_test = np.load('./datasets/kmnist-test-labels.npz')['arr_0']

t_train = np.identity(10)[t_train]
t_test = np.identity(10)[t_test]
x_train = x_train.reshape((60000,-1))
x_test = x_test.reshape((10000,-1))
x_train = x_train/255
x_test = x_test/255

In [ ]:
net = Classification(
      learning_rate=0.05,
      epoch=30000,
      batch_size=500,
      optimize_initial_weight = True,
      optimizer = 'momentum',
      log_frequency = 100,
      loss_func = 'square'
)
net.activation['output'] = 'sigmoid'
layer_list = [784,[500,80],10]
net.set_layer(layer_list)

<< successfully layers are updated >>


In [ ]:
elapsed_time,train_acc = net.train(x_train,t_train)

--------- epoch0 ---------
loss : 646.936331320113
accuracy : 0.074
time : 0.026880979537963867 [sec]
--------------------------

--------- epoch100 ---------
loss : 210.88891245090602
accuracy : 0.428
time : 4.709453821182251 [sec]
----------------------------

--------- epoch200 ---------
loss : 177.8458563528766
accuracy : 0.576
time : 8.594067811965942 [sec]
----------------------------

--------- epoch300 ---------
loss : 133.97213317872473
accuracy : 0.702
time : 14.129971981048584 [sec]
----------------------------

--------- epoch400 ---------
loss : 112.94848707587985
accuracy : 0.758
time : 18.514842987060547 [sec]
----------------------------

--------- epoch500 ---------
loss : 104.98429834472309
accuracy : 0.738
time : 22.692945957183838 [sec]
----------------------------

--------- epoch600 ---------
loss : 86.62983863541365
accuracy : 0.792
time : 26.436879873275757 [sec]
----------------------------

--------- epoch700 ---------
loss : 90.46661084139173
accuracy : 0.75


In [ ]:
print('train accuracy : {}\ntest accuracy : {}'.format(
    net.accuracy(x_train,t_train),net.accuracy(x_test,t_test)))

In [ ]:
net.visualize()

In [ ]:
net.save('anyname')

In [ ]:
new_net = Classification.load('anyname')
new_net.accuracy(x_test,t_test)